# Day 20 - Grove Positioning System

>It's finally time to meet back up with the Elves. When you try to contact them, however, you get no reply. Perhaps you're out of range?
>
>You know they're headed to the grove where the star fruit grows, so if you can figure out where that is, you should be able to meet back up with them.
>
>Fortunately, your handheld device has a file (your puzzle input) that contains the grove's coordinates! Unfortunately, the file is encrypted - just in case the device were to fall into the wrong hands.
>
>Maybe you can decrypt it?
>
>When you were still back at the camp, you overheard some Elves talking about coordinate file encryption. The main operation involved in decrypting the file is called mixing.
>
>The encrypted file is a list of numbers. To mix the file, move each number forward or backward in the file a number of positions equal to the value of the number being moved. The list is circular, so moving a number off one end of the list wraps back around to the other end as if the ends were connected.
>
>For example, to move the 1 in a sequence like 4, 5, 6, 1, 7, 8, 9, the 1 moves one position forward: 4, 5, 6, 7, 1, 8, 9. To move the -2 in a sequence like 4, -2, 5, 6, 7, 8, 9, the -2 moves two positions backward, wrapping around: 4, 5, 6, 7, 8, -2, 9.
>
>The numbers should be moved in the order they originally appear in the encrypted file. Numbers moving around during the mixing process do not change the order in which the numbers are moved.
>
>Consider this encrypted file:
>```
>1
>2
>-3
>3
>-2
>0
>4
>```
>Mixing this file proceeds as follows:
>```
>Initial arrangement:
>1, 2, -3, 3, -2, 0, 4
>
>1 moves between 2 and -3:
>2, 1, -3, 3, -2, 0, 4
>
>2 moves between -3 and 3:
>1, -3, 2, 3, -2, 0, 4
>
>-3 moves between -2 and 0:
>1, 2, 3, -2, -3, 0, 4
>
>3 moves between 0 and 4:
>1, 2, -2, -3, 0, 3, 4
>
>-2 moves between 4 and 1:
>1, 2, -3, 0, 3, 4, -2
>
>0 does not move:
>1, 2, -3, 0, 3, 4, -2
>
>4 moves between -3 and 0:
>1, 2, -3, 4, 0, 3, -2
>```
>Then, the grove coordinates can be found by looking at the 1000th, 2000th, and 3000th numbers after the value 0, wrapping around the list as necessary. In the above example, the 1000th number after 0 is 4, the 2000th is -3, and the 3000th is 2; adding these together produces 3.
>
>Mix your encrypted file exactly once. What is the sum of the three numbers that form the grove coordinates?

## Read input

Since today's puzzle requires shifting values around **and** remembering the original order **and** the list contains duplicate numbers, I read the data in into a list of Number tuples that in addition to its value, knows its id (original index).

In [11]:
from utils import read_input
from collections import namedtuple

Number = namedtuple('Number', ['id', 'value'])

data = read_input(20, int)
data = [Number(i, value) for i, value in enumerate(data)]

To mix the numbers, I create a duplicate list to remember the original order. I then shuffle values around and replace the data list to put the value into the same position.

This is not very memory efficient as each loop, I have three lists of the same size at memory: the order, the old data and the new data. But for an input of 5000 items, it's not an issue. If the puzzle would be exponentially larger, this could slow it down and another solution should be made.

To calculate the end solution, I find the first Number with value 0 and then move 1000, 2000 and 3000 steps forward from that and sum them together.

In [13]:
def mix(data):
    order = data[:]
    
    for number in order:
        d_idx = data.index(number)
        new_idx = (d_idx + number.value) 
        new_idx = new_idx % (len(data)-1)
        data.remove(number)
        
        data = data[:new_idx] + [number] + data[new_idx:]
    
    return data       

def grove_coordinates(data):
    zero_index = None
    for i, number in enumerate(data):
        if number.value == 0:
            zero_index = i
            break
    
    first = data[(zero_index + 1000) % len(data)]
    second = data[(zero_index + 2000) % len(data)]
    third = data[(zero_index + 3000) % len(data)]
    
    return first.value + second.value + third.value

In [14]:
result = grove_coordinates(mix(data.copy()))
print(f'Part 1: {result}')
assert result == 23321

Part 1: 23321


## Part 2

>The grove coordinate values seem nonsensical. While you ponder the mysteries of Elf encryption, you suddenly remember the rest of the decryption routine you overheard back at camp.
>
>First, you need to apply the decryption key, 811589153. Multiply each number by the decryption key before you begin; this will produce the actual list of numbers to mix.
>
>Second, you need to mix the list of numbers ten times. The order in which the numbers are mixed does not change during mixing; the numbers are still moved in the order they appeared in the original, pre-mixed list. (So, if -3 appears fourth in the original list of numbers to mix, -3 will be the fourth number to move during each round of mixing.)
>
>Using the same example as above:
>```
>Initial arrangement:
>811589153, 1623178306, -2434767459, 2434767459, -1623178306, 0, 3246356612
>
>After 1 round of mixing:
>0, -2434767459, 3246356612, -1623178306, 2434767459, 1623178306, 811589153
>
>After 2 rounds of mixing:
>0, 2434767459, 1623178306, 3246356612, -2434767459, -1623178306, 811589153
>
>After 3 rounds of mixing:
>0, 811589153, 2434767459, 3246356612, 1623178306, -1623178306, -2434767459
>
>After 4 rounds of mixing:
>0, 1623178306, -2434767459, 811589153, 2434767459, 3246356612, -1623178306
>
>After 5 rounds of mixing:
>0, 811589153, -1623178306, 1623178306, -2434767459, 3246356612, 2434767459
>
>After 6 rounds of mixing:
>0, 811589153, -1623178306, 3246356612, -2434767459, 1623178306, 2434767459
>
>After 7 rounds of mixing:
>0, -2434767459, 2434767459, 1623178306, -1623178306, 811589153, 3246356612
>
>After 8 rounds of mixing:
>0, 1623178306, 3246356612, 811589153, -2434767459, 2434767459, -1623178306
>
>After 9 rounds of mixing:
>0, 811589153, 1623178306, -2434767459, 3246356612, 2434767459, -1623178306
>
>After 10 rounds of mixing:
>0, -2434767459, 1623178306, 3246356612, -1623178306, 2434767459, 811589153
>```
>The grove coordinates can still be found in the same way. Here, the 1000th number after 0 is 811589153, the 2000th is 2434767459, and the 3000th is -1623178306; adding these together produces 1623178306.
>
>Apply the decryption key and mix your encrypted file ten times. What is the sum of the three numbers that form the grove coordinates?

My first impression seeing the part 2 description: 😱. 811589153 looked like such a huge number but once I realized that it does not affect the amount of steps taken nor the size of the lists, I realized that it does not increase the complexity of my solution at all.

All I had to do, was to calculate a new starting point by multiplying each value with the key and then run the mixing ten times.

In [15]:
def mix2(data):
    data = [Number(num.id, num.value * 811589153) for num in data]
    order = data[:]
    
    for cycle in range(10):
        for number in order:
            d_idx = data.index(number)
            new_idx = (d_idx + number.value) 
            new_idx = new_idx % (len(data)-1)
            data.remove(number)

            data = data[:new_idx] + [number] + data[new_idx:]
    
    return data

In [17]:
result2 = grove_coordinates(mix2(data.copy()))
print(f'Part 2: {result2}')
assert result2 == 1428396909280

Part 2: 1428396909280
